# Strojenie parametrów - czyli jak znaleźć najlepszy model?

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
np.random.seed(123) 
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
r'''COLOR = 'white'
matplotlib.rcParams['text.color'] = COLOR
matplotlib.rcParams['axes.labelcolor'] = COLOR
matplotlib.rcParams['xtick.color'] = COLOR
matplotlib.rcParams['ytick.color'] = COLOR
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['font.size'] = 14'''

### Wczytanie danych i podział na train-test

In [ ]:
data=pd.read_csv('heart.csv')

y = np.array(data['chd'])
X = data.drop(['chd'],axis=1)

map_dict = {'Present': 1, 'Absent':0}
X['famhist'] = X['famhist'].map(map_dict)
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.2)

### Drzewo decyzyjne

In [ ]:
from sklearn.tree import DecisionTreeClassifier
#parametry domyślne
tree_model= DecisionTreeClassifier()

tree_model.fit(X_train, y_train).score(X_test, y_test)

## Po co to wszystko?

![image.png](http://snoek.ddns.net/~oliver/mysite/images/blog7_accuracyprecision_banner.png)
http://snoek.ddns.net/~oliver/mysite/the-bias-variance-tradeoff.html

### Jak zdiagnozować problem? Learning Curves

In [ ]:
from sklearn.model_selection import learning_curve
tree_model= DecisionTreeClassifier(max_depth=5)
train_sizes, train_scores, test_scores = \
    learning_curve(tree_model, X_train[:300], y_train[:300], train_sizes=np.linspace(0.1, 1, 100),
                   scoring="accuracy", cv=3)

fig = plt.figure()
ax = fig.add_subplot(111)

plt.plot(train_sizes, test_scores.mean(1), 'o-', color="r",
         label="test")
plt.plot(train_sizes, train_scores.mean(1), 'o-', color="b",
         label="train")


plt.xlabel("Train size")
plt.ylabel("Mean Squared Error")
plt.title('Learning curves')
plt.legend(loc="best")
l = ax.legend()
for text in l.get_texts():
    text.set_color("black")

plt.show()

### Diagnoza: High Variance

In [ ]:
from sklearn.model_selection import learning_curve
tree_model= DecisionTreeClassifier(max_depth=1)
train_sizes, train_scores, test_scores = \
    learning_curve(tree_model, X_train[:300], y_train[:300], train_sizes=np.linspace(0.1, 1, 100),
                   scoring="accuracy", cv=3)

fig = plt.figure()
ax = fig.add_subplot(111)

plt.plot(train_sizes, test_scores.mean(1), 'o-', color="r",
         label="test")
plt.plot(train_sizes, train_scores.mean(1), 'o-', color="b",
         label="train")


plt.xlabel("Train size")
plt.ylabel("Mean Squared Error")
plt.title('Learning curves')
plt.legend(loc="best")
l = ax.legend()
for text in l.get_texts():
    text.set_color("black")

plt.show()

### Powyżej mamy High Bias
Andrew Ng: https://www.youtube.com/watch?v=ISBGFY-gBug&t=344s

### Kroswalidacja

In [ ]:
import sklearn
from sklearn.model_selection import cross_val_score
tree_model= DecisionTreeClassifier()
results=cross_val_score(tree_model, X, y) # można zdefiniować: scoring='roc_auc'
# uwaga: tutaj nie strojono parametrów więc można użyć całego zbioru
print(np.mean(results), np.std(results))

In [ ]:
# mamy wiele możliwych metryk
sklearn.metrics.SCORERS.keys()

## Przeszukiwanie przestrzeni parametrów

### Grid Search
Klasyczne przeszukiwanie wszystkich możliwości

#### Zalety:
* sprawdzimy wszystkie kombinacje

#### Wady:
* musimy wiedzieć jakie konkretnie kombinacje chcemy sprawdzić
* długo trwa

In [ ]:
from sklearn.model_selection import GridSearchCV

max_depth=[3, 5, 6]
criterion=["gini","entrophy"]
ccp_alpha=[0, 0.05]
param_grid = dict(max_depth=max_depth,criterion=criterion, ccp_alpha=ccp_alpha)

In [ ]:
tree_model= DecisionTreeClassifier()
grid = GridSearchCV(estimator=tree_model, param_grid=param_grid, cv = 3, n_jobs=-1)

grid_result = grid.fit(X_train, y_train) #tutaj lepiej zastosować tylko trainset

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
grid_result.best_estimator_.score(X_test, y_test)

### Random Search 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random = RandomizedSearchCV(estimator=tree_model, param_distributions=param_grid, cv = 3, n_jobs=-1)

random_result = random.fit(X_train, y_train)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

In [ ]:
best_model=random_result.best_estimator_
best_model.score(X_test, y_test)

Zauważmy, że tu mało się zmienia. Nadal przeszukujemy siatkę, ale w sposób losowy. Nasza siatka jest dość mała, więc i tak sprawdzamy wszystkie możliwości

## Zdefiniowanie rozkładów

<div>
<img src="https://ksopyla.com/wp-content/uploads/2018/12/Grid_search_vs_random_search_cross_validation.png" width="800"/>
</div>

sos: https://ksopyla.com/machine-learning/grid-random-search-scikit-learn-dobor-parametrow/

In [ ]:
from scipy.stats import poisson,expon
param_grid = {'ccp_alpha': expon(0.08),
             'max_depth': poisson(5)}

In [ ]:
random = RandomizedSearchCV(estimator=tree_model, param_distributions=param_grid, cv = 3, n_jobs=-1, random_state=123)

random_result = random.fit(X_train, y_train)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

## Bayes optimization
Bardziej inteligentne przeszukiwanie na podstawie Gaussian process

In [ ]:
#!pip install scikit-optimize
#!pip install --upgrade scikit-learn==0.23.2
#Jest problem z najnowszą wersją sklearn

In [ ]:
from skopt import BayesSearchCV
from sklearn.svm import SVC

opt = BayesSearchCV(
    SVC(),
    {
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=8, # tu powinno być więcej, ale to się długo liczy
    cv=3
)

opt.fit(X_train, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))

Przykład pochodzi stąd: https://scikit-optimize.github.io/stable/auto_examples/sklearn-gridsearchcv-replacement.html  
Tam też jest więcej parametrów

# Są narzędzia, które robią wszystko za nas

## automl from sklearn
(działa tylko na Linuxie - rozwiązanie: korzystać z Google Colaboratory)  
Nie ma na Anacondzie, wymaga sklearn >= 0.24

In [ ]:
#import autosklearn.classification
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_pred = automl.predict(X_test)
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_pred))

Jak widać trudno się czasem dogadać ze wszystkimi pakietami...

### TPOT
https://epistasislab.github.io/tpot/  
Korzysta z algorytmów genetycznych

![image.png](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1537396029/output_2_0_d7uh0v.png)
https://www.datacamp.com/community/tutorials/tpot-machine-learning-python

In [ ]:
# ! pip install tpot

In [ ]:
from tpot import TPOTClassifier
#from tpot import TPOTRegressor

tpot = TPOTClassifier(generations=5,verbosity=2)

tpot.fit(X_train, y_train)

In [ ]:
tpot.score(X_test, y_test)

### Potencjalne wady:
- długo się liczy
- nie gwarantuje zbieżności

Trochę więcej (np. o parametrach) można poczytać tutaj https://www.datacamp.com/community/tutorials/tpot-machine-learning-python

## Selekcja zmiennych

### Filtry
Najprostsze metody, nie zależą od moeli. To jest preprocessing.

Przykłady:
- korelacja ze zmienną celu
- informacja wzajema ze zmienną celu (VIF)

## VIF
Variance Inflation Factor  
Dopasowujemy model regresji liniowej, gdzie jedna kolumna jest targetem, a pozostałe feature'ami

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
pd.DataFrame(data = [(X_train.columns[i], vif(X_train.values, i)) for i in range(X_train.shape[1])], columns=['feature', 'vif_score'])

## Metody wbudowane
czyli takie metody, które są wbudowane w algorytm - Lasso albo Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
rf.fit(X_train, y_train)

In [ ]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]


plt.figure()
plt.title("Feature importance")
plt.bar(X.columns, importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(rotation=45)
plt.show()

### Wrappery
Oceniają poszczególne zbiory zmiennych na podstawie wybranych metryk (accuracy, AUC, ...). Zazwyczaj działa to w ten sposób, że iteracyjnie dodajemy/odejmujemy kolejne zmienne aż osiągniemy daną liczbę zmiennych/wynik modelu.   

Zasadniczo to taki model, który ocenia zmienne. Też trzeba go zfitować.

Przykłady:
- Forward Selection (K Best, Select From Model) 
- Recursive Feature Elimination
- Boruta

### SelectKBest 
Wybieramy K Najlepszych cech na podstawie zadanego kryterium  
Default: na podstawie algorytmu ANOVA

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
tree_model=DecisionTreeClassifier()

pipe = Pipeline([
    ('select', SelectKBest()),
    ('model', tree_model)])

k=[5,6]
max_depth=[3, 5, 6]
criterion=["gini","entrophy"]

# uwaga: gdy podajemy parametry do strojenia gdy mamy pipeline to trzeba w nazwach kluczy podać nazwę_danego_etapu__ 
# (poprzedzoną dwoma podkreślnikami)
param_grid = {"model__max_depth": max_depth, "model__criterion": criterion, "select__k": k}

search = GridSearchCV(pipe, param_grid, cv=5).fit(X_train, y_train)

In [ ]:
search.score(X_test, y_test)

In [ ]:
X_train.columns[search.best_estimator_.steps[0][1].get_support()]

### Select From Model

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
model_selector = SelectFromModel(
    LogisticRegression(C=0.000025, solver="liblinear"),
    threshold = "mean"
)

pipe = Pipeline([
    ('select', model_selector),
    ('model', tree_model)])

penalty = ['l1', 'l2']

# tu odwołujemy się do select (SelectFromModel) i do estymatora w środku (LR), więc dwa razy podwójny podkreślnik (__)
param_grid = {"model__max_depth": max_depth, "model__criterion": criterion, "select__estimator__penalty": penalty}
search = GridSearchCV(pipe, param_grid, cv=5).fit(X_train, y_train)

In [ ]:
X_train.columns[search.best_estimator_.steps[0][1].get_support()]

### Recursive Feature Elimination
Zasadniczo, działa tak jak brzmi. 
Dopasuj model, zobacz, która cecha jest "najmniej ważna".  
Powtórz dla modelu bez tej cechy.  

Najmniej ważna to może być względem *feature_importances* albo *coef*

In [ ]:
from sklearn.feature_selection import RFE
estimator = tree_model
selector = RFE(estimator, n_features_to_select=3, step=1) #step ile (procent) cech usuwamy w kroku
selector = selector.fit(X_train, y_train) 

In [ ]:
print(selector.support_)
selector.ranking_

### Boruta
Próbuje znaleźć *wszystkie* cechy, które mają związek z targetem

In [ ]:
#!pip install boruta

In [ ]:
from boruta import BorutaPy
# for classification only (we need to convert pd.DataFrame to np.array)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

feat_selector.fit(X_train.values, y_train)
feat_selector.support_
feat_selector.ranking_

# wybór odpowiednich zmiennych ze zbioru testowego
X_filtered = feat_selector.transform(X_test.values)

In [ ]:
X_train.columns[feat_selector.support_]

## Ciekawostki: 
Przykład wizualizacji dla kilku metryk 
- https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html   

Jak robić strojenie parametrów dla różnych modeli?
- http://www.davidsbatista.net/blog/2018/02/23/model_optimization/
- https://stackoverflow.com/questions/50265993/alternate-different-models-in-pipeline-for-gridsearchcv  

Inny przykład automl
- Hyperopt https://hyperopt.github.io/hyperopt/

Paulina wrzuciła ten link a propos metod wbudowanych, więc nie chciałem być gorszy
- https://towardsdatascience.com/the-mathematics-of-decision-trees-random-forest-and-feature-importance-in-scikit-learn-and-spark-f2861df67e3